In [ ]:
#@title <font color=deepskyblue><b>Run this to set up your environment! (Ignore errors!) </b></font>
import warnings
warnings.filterwarnings('ignore')

from google.colab import files
from IPython.display import HTML, Image
import zipfile

!yes | pip install streamlit-drawable-canvas --quiet &> /dev/null

#For streamlit
!pip install protobuf==4.21.1 &> /dev/null
!pip -q install streamlit &> /dev/null
!pip -q install pyngrok &> /dev/null

import io
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image
import os
import gdown


def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def capture_image(filename):
  try:
    take_photo(filename)
    print('Saved to {}'.format(filename))

    # Show the image which was just taken.
    display(Image(filename))
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

import numpy as np
import tensorflow as tf
import cv2
from google.colab.patches import cv2_imshow


In [ ]:
#@title Run this cell to upload and extract your model into Colab (make sure that your file has the name `converted_keras.zip`!)

version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
  import tf_keras as keras
else:
  keras = tf.keras

# upload model zip file
uploaded = files.upload()

# extract model
np.set_printoptions(suppress=True)
try:
  file_name = "converted_keras.zip"
  print("Extracting model...")
  with zipfile.ZipFile(file_name, 'r') as zip_ref:
      zip_ref.extractall('')

  class_names = np.genfromtxt("labels.txt", dtype="str", delimiter='\n')
  for i in range(len(class_names)):
    class_names[i] = ' '.join(class_names[i].split(' ')[1::])

  predictions = [0.0] * len(class_names)

  model = keras.models.load_model('keras_model.h5', compile=False)
  print("Success! Model Extracted!")
except (IndexError, NameError):
    print("Oops! Cannot find file to unzip. Please try uploading a zip file for your model")

def get_image_classification():
  image_name = 'test.jpg'
  image = cv2.imread(image_name)
  cv2_imshow(image)
  return classify_image(image_name)['classification']

def get_image_predictions():
  return classify_image('test.jpg')['predictions']

def get_best_prediction():
  return max(get_image_predictions())

def classify_image(image_name):
  image = cv2.imread(image_name)
  image = cv2.resize(image,(224,224)).astype('float32')

  image = np.reshape(image,(1,224,224,3))
  image = image / 255

  predictions = model.predict(image)[0]

  pred_class = class_names[np.argmax(predictions)]

  return {'classification': pred_class, 'predictions': predictions}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
capture_image("test.jpg")

In [ ]:
classification = get_image_classification()
print(classification)

In [ ]:
#@title Test Sketch and Obtain Classification:


canvas_html = """
<canvas width=%d height=%d></canvas>
<br>
<button>Save Image</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
ctx.fillStyle = "#eeeeee";
ctx.fillRect(0, 0, canvas.width, canvas.height);
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def get_image_classification():
  image_name = 'test.png'
  image = cv2.imread(image_name)
  # cv2.imshow(image)
  return classify_image(image_name)['classification']

def get_image_predictions():
  return classify_image('test.png')['predictions']

def get_best_prediction():
  return max(get_image_predictions())

def classify_image(image_name):
  image = cv2.imread(image_name)
  image = cv2.resize(image,(224, 224)).astype('float32')

  image = np.reshape(image,(1,224, 224 ,3))
  # image = image / 255

  predictions = model.predict(image)[0]
  class_list = ['Triangle','Circle','Square']
  pred_class = class_list[np.argmax(predictions)]

  return {'classification': pred_class, 'predictions': predictions}

def test_sketch(w = 224, h = 224, line_width = 1):
  path = ''
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  filename = 'test.png'
  with open(os.path.join(path, filename), 'wb') as f:
    f.write(binary)
  print("Saved Image")

test_sketch()

import seaborn as sns
import numpy as np
import cv2
classification = get_image_classification()
predictions = get_image_predictions()
get_image_classification()

class_list = ['Triangle','Circle','Square']
print("classification: " + classification)


In [ ]:
###YOUR CODE HERE

In [ ]:
###YOUR CODE HERE

In [ ]:
predictions = get_image_predictions()
print(predictions)

In [ ]:
###YOUR CODE HERE###

In [ ]:
print(predictions)
print(class_names)

plt.ylim(0.00, 1.00)

###YOUR CODE HERE

chart = sns.barplot(x = None, y = None) #REPLACE THE 'None' WITH THE CORRECT DATA FOR X AND Y
chart.set(xlabel="FILL ME IN", ylabel = "FILL ME IN") #FILL IN THE X-AXIS AND Y-AXIS LABELS

####


plt.show()


In [ ]:
%%writefile header.py
import streamlit as st
import streamlit.components.v1 as components

def create_header():
  # Create a title
  # Create a header
  # Create a subheader
  pass

In [ ]:
%%writefile canvas_size.py

def get_canvas_size():
  #CHANGE THE WIDTH AND HEIGHT
  width = 1
  height = 1
  return width, height

In [ ]:
%%writefile response.py
import pandas as pd
import streamlit as st
import streamlit.components.v1 as components

def get_app_response(classification, probability):
  if classification == "FILL ME IN":
    st.write("FILL ME IN")
  #ADD MORE IF / ELIF STATEMENTS HERE
  else:
    st.write("FILL ME IN")

In [ ]:
#@title Run this to set up your model in your app!
%%writefile classifier.py
import streamlit as st
import streamlit.components.v1 as components

import numpy as np
import tensorflow as tf
import cv2

import tf_keras as keras

class_names = np.genfromtxt("labels.txt", dtype="str", delimiter='\n')
for i in range(len(class_names)):
  class_names[i] = ' '.join(class_names[i].split(' ')[1::])
np.set_printoptions(suppress=True)

model = keras.models.load_model('keras_model.h5', compile = False)

def get_image_classification(image_data):
    img = image_data
    img32 = np.float32(img)
    bgr = cv2.cvtColor(img32, cv2.COLOR_BGRA2BGR)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    rgb = cv2.resize(rgb,(224,224)).astype('float32')
    rgb = np.reshape(rgb,(1,224,224,3))
    rgb = rgb / 255

    predictions = model.predict(rgb)[0]
    classification = class_names[np.argmax(predictions)]
    return classification, predictions